# apply registration from on other channels!

In [19]:

%load_ext autoreload
%autoreload 2
import sys
from tifffile import imwrite as tif_imwrite
from tifffile import imread as tif_imread
from IPython.display import display, HTML
import numpy as np
from pathlib import Path
import zarr
display(HTML("<style>.container { width:100% !important; }</style>"))
import sys
# Path for bigstream unless you did pip install
sys.path = ["/mnt/nasquatch/data/2p/jonna/Code_Python/Notebooks_Jonna/BigStream/bigstream_github"] + sys.path 
sys.path.append("../../")
from bigstream.align import alignment_pipeline
from bigstream.piecewise_align import distributed_piecewise_alignment_pipeline
from bigstream.transform import apply_transform
from bigstream.piecewise_transform import distributed_apply_transform
from registrations_funcs import verify_rounds

from tqdm.auto import trange
import os
from pathlib import Path
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import hjson
hjson_file = fr'/home/oamsalem/code/Projects/EZfish/ezfish_pipeline/examples/RG026.hjson'



manifest = hjson.load(open(hjson_file, 'r'))

round_to_rounds, reference_round = verify_rounds(manifest, registered_paths=True)
round_to_rounds

All files found
Rounds available for registerion apply: 02 03 

{'02': OrderedDict([('round', '02'),
              ('channels', ['DAPI', 'GLP1R', 'CHAT', 'TAC1']),
              ('resolution', [1.515, 1.515, 5]),
              ('image_path',
               PosixPath('/mnt/bananas/data/ezfish/jonna/RG026/HCR/RG026_HCR02_To_HCR01.tif')),
              ('registrations',
               ['local_params_scan_nodeform/rz1_rxy25_bs3to30_mt0.2/bs50_50_12_1_15_bs5to70_mt0.2'])]),
 '03': OrderedDict([('round', '03'),
              ('channels', ['DAPI', 'RORB', 'NPR3', 'VGAT']),
              ('resolution', [1.515, 1.515, 5]),
              ('image_path',
               PosixPath('/mnt/bananas/data/ezfish/jonna/RG026/HCR/RG026_HCR03_To_HCR01.tif')),
              ('registrations',
               ['local_params_scan_nodeform/rz1_rxy25_bs3to30_mt0.2/bs50_50_12_1_15_bs5to70_mt0.2'])])}

### PARAMETERS

In [42]:
# Choose the round to register
HCR_round_to_register = '03'# 02,03...?

assert HCR_round_to_register in round_to_rounds.keys(), f"round {HCR_round_to_register} not found in rounds to register"
assert 'registrations' in round_to_rounds[HCR_round_to_register], f"round {HCR_round_to_register} was not found in params.hjson"



In [52]:
mov_round = round_to_rounds[HCR_round_to_register]
# File names
HCR_fix_image_path = reference_round['image_path'] # The fix image that all other rounds will be registerd to (include all channels!)
HCR_mov_image_path = mov_round['image_path'] # The image that will be registered to the fix image (include all channels!)

round_folder_name = f"HCR{HCR_round_to_register}_to_HCR{reference_round['round']}"

# resolution of the images
fix_image_spacing = np.array(reference_round['resolution']) # Y,X,Z
mov_image_spacing = np.array(mov_round['resolution']) # Y,X,Z


# spatial downsampling, probably no need to test. (Changed x and y from 3 to 2 for CIM round 5)
red_mut_x = manifest['HCR_to_HCR_params']['red_mut_x']
red_mut_y = manifest['HCR_to_HCR_params']['red_mut_y']
red_mut_z = manifest['HCR_to_HCR_params']['red_mut_z']

fix_lowres_spacing = fix_image_spacing * np.array([red_mut_y,red_mut_x,red_mut_z])
mov_lowres_spacing = mov_image_spacing * np.array([red_mut_y,red_mut_x,red_mut_z])



In [51]:
# get block size from the registration file
blocksize_match = re.findall(r'bs(\d+)_(\d+)_(\d+)', Path(round_to_rounds[HCR_round_to_register]['registrations'][0]).name)
blocksize = [int(num) for num in blocksize_match[0]]

HCR_fix_round = tif_imread(HCR_fix_image_path)[:,0]
HCR_mov_round = tif_imread(HCR_mov_image_path)



### Image parameters

In [55]:


reg_path =  Path(manifest['base_path']) / manifest['mouse_name'] / 'OUTPUT' / 'HCR' / 'registrations'/ round_folder_name / round_to_rounds[HCR_round_to_register]['registrations'][0]
full_stack_path =  Path(manifest['base_path']) / manifest['mouse_name'] / 'OUTPUT' / 'HCR' 
affine = np.loadtxt(fr"{reg_path}/_affine.mat")
deform = zarr.load(fr"{reg_path}/deform.zarr")

data_paths = []
#Loop through channels starting with 1, which ignores the first channel which has already been registered
for channel in trange(1,HCR_mov_round.shape[1]):
    
    if 1:#try:
        fix_highres = HCR_fix_round.transpose(2,1,0) # from Z,X,Y to Y,X,Z
        
        HCR_mov_round_C = HCR_mov_round[:,channel]
        # Move Image
        mov_highres = HCR_mov_round_C.transpose(2,1,0)




        # apply the transforms
        print(os.listdir(reg_path))

        local_aligned = distributed_apply_transform(
            fix_highres, mov_highres,
            fix_image_spacing, mov_image_spacing,
            transform_list=[affine, deform],
            blocksize=blocksize,
            write_path=fr"{reg_path}/out_c{channel}.zarr")
        print(fr'saved output {reg_path}/out_c{channel}.zarr')

        data = zarr.load(fr"{reg_path}/out_c{channel}.zarr")
        data_paths.append(fr"{reg_path}/out_c{channel}.zarr")
        tif_imwrite(fr"{reg_path}/out_c{channel}.tiff"
                    ,data.transpose(2,1,0)
                    )
full_stack = [] 
for path in data_paths:
    full_stack.append(zarr.load(path))
full_stack = np.stack(full_stack)
tif_imwrite(full_stack_path / f"{round_folder_name}.tiff"
            ,full_stack.transpose(3,2,1,0)
            )

#         "A:\ezfish_test_delete\RG23_SLICE002\RG23_SLICE2_HCR1.tif"

  0%|          | 0/3 [00:00<?, ?it/s]

['deform.zarr', '_mov_lowres_affine.tiff', '_affine.mat', '_affine.npy', '_fix_lowres.tiff', '_mov_lowres.tiff', '_both.tiff']
{'host': ''}


/mnt/colab/colab_shared/anaconda3/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35037 instead
  warnings.warn(


Cluster dashboard link:  http://10.35.76.107:35037/status


2024-08-16 14:45:36,081 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-08-16 14:45:59,239 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-08-16 14:46:10,723 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-08-16 14:46:17,231 - distributed.worker - ERROR - Exception during execution of task ('store-map-2fe4fbba2ea7be595000880daafa3fde', 1, 37, 0).
Traceback (most recent call last):
  File "/mnt/colab/colab_shared/anaconda3/lib/python3.9/site-packages/distributed/worker.py", line 2341, in _prepare_args_for_execution
    data[k] = self.data[k]
  File "/mnt/colab/colab_shared/anaconda3/lib/python3.9/site-packages/zict/buffer.py", line 87, in __getitem__
    raise KeyError(key)
KeyError: "('getitem-transform_single_block-getitem-store-map-2fe4fbba2ea7be595000880daafa3fde', 1, 37, 0)"

During handling of t

KeyboardInterrupt: 

In [33]:
'registrations' in round_to_rounds['03']

True

### PARAMETERS

In [ ]:
# Choose the round to register
HCR_round_to_register = '03'# 02,03...?

assert HCR_round_to_register in round_to_rounds.keys(), f"round {HCR_round_to_register} not found in rounds to register"

